In [1]:
%run init_notebook.py

PROJECT ROOT is: c:\Users\Articuno\Desktop\TFG-info


In [ ]:

%run init_notebook.py

import torch
import torch.nn as nn
import torchaudio

# Adjust these imports according to your project structure:
from src.models import AutoEncoder
from src.config import CONV_KERNEL_SIZE, CONV_STRIDE, CONV_PADDING
from src.utils.models import compute_output_size, compute_flattened_size

# Define parameters
input_height = 64
input_width = 128
latent_dim = 30
in_channels = 1
filters = [32, 64, 128]

# Instantiate the autoencoder model.
# Make sure your AutoEncoder class is defined to accept (input_height, input_width, latent_dim, in_channels, filters)
model = AutoEncoder(input_height, input_width, latent_dim, in_channels, filters)
print("AutoEncoder model:")
print(model)

# Create a dummy input tensor: shape [batch_size, in_channels, input_height, input_width]
batch_size = 4
dummy_input = torch.randn(batch_size, in_channels, input_height, input_width)

# Pass the dummy input through the autoencoder d
output = model(dummy_input)

# Print input and output shapes
print("Input shape:", dummy_input.shape)
print("Output shape:", output.shape)

# Check if the output shape matches the input shape
if dummy_input.shape == output.shape:
    print("Success: Output shape matches input shape.")
else:
    print("Mismatch: Adjust output_padding in your decoder layers if necessary.")


AutoEncoder model:
AutoEncoder(
  (encoder): Encoder(
    (encoder): Sequential(
      (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): ReLU()
      (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (3): ReLU()
      (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (5): ReLU()
      (6): Flatten(start_dim=1, end_dim=-1)
      (7): Linear(in_features=16384, out_features=30, bias=True)
    )
  )
  (decoder): Decoder(
    (decoder): Sequential(
      (0): Linear(in_features=30, out_features=16384, bias=True)
      (1): Unflatten(dim=1, unflattened_size=(128, 8, 16))
      (2): ConvTranspose2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
      (3): ReLU()
      (4): ConvTranspose2d(128, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
      (5): ReLU()
      (6): ConvTranspose2d(64, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1),

In [1]:
%run init_notebook.py

import torch
import torch.nn as nn
import torch.optim as optim
import torchaudio.transforms as T
import librosa
import soundfile as sf
import time
from tqdm import tqdm
from torch.utils.data import DataLoader, Subset
from torch.optim.lr_scheduler import ReduceLROnPlateau
from IPython.display import Audio, display

from src.dataset import NSynth   
from src.models import AutoEncoder  
from src.config import CONV_KERNEL_SIZE, CONV_STRIDE, CONV_PADDING
from src.utils.dataset import load_raw_waveform

# Mel spectogram definition
mel_transform = nn.Sequential(
    T.MelSpectrogram(
        sample_rate=16000,
        n_fft=1024,
        hop_length=501,
        n_mels=64,
        # power=2.0,      
        normalized=True
    ),
    # T.AmplitudeToDB(stype="power")  # convert power -> dB
)

# Datasets and DataLoaders
train_dataset = NSynth(partition='training', transform=mel_transform)
valid_dataset = NSynth(partition='validation', transform=mel_transform)
test_dataset  = NSynth(partition='testing', transform=mel_transform)

# Subset for quicker training
train_dataset = Subset(train_dataset, list(range(50000)))

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=8, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=True, num_workers=8, pin_memory=True)
test_loader  = DataLoader(test_dataset, batch_size=1, shuffle=False)

# Model, Optimizer, and Loss
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_height = 64
input_width  = 128
latent_dim   = 64
in_channels  = 1
filters      = [64, 128, 256]

model = AutoEncoder(input_height, input_width, latent_dim, in_channels, filters).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
# scheduler = ReduceLROnPlateau(optimizer, factor=0.5, patience=5, verbose=True)
criterion = nn.MSELoss()

# Training Loop
num_epochs = 50
log_interval = 10

print(f"Starting training on {device}...")
for epoch in tqdm(range(num_epochs), desc="Training"):
    model.train()
    start_epoch_time = time.time()
    train_loss = 0.0
    
    # Only interested in mel_spec from (waveform, sample_rate, key, metadata)
    for i, (mel_spec, _, _, _) in enumerate(train_loader):
        mel_spec = mel_spec.to(device)  # shape [batch_size, 1, 64, 128]

        optimizer.zero_grad()
        output = model(mel_spec)
        loss = criterion(output, mel_spec)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * mel_spec.size(0)

        # if (i + 1) % log_interval == 0:
        #     print(f"  [Epoch {epoch+1}, Step {i+1}/{len(train_loader)}] loss: {loss.item():.4f}")

    # Compute average epoch loss
    train_loss /= len(train_loader.dataset)
    epoch_time = time.time() - start_epoch_time

    # model.eval()
    val_loss = 0.0
    # with torch.no_grad():
    #     for (mel_spec, _, _, _) in valid_loader:
    #         mel_spec = mel_spec.to(device)  # shape [batch_size, 1, 64, 128]
    #         output = model(mel_spec)
    #         loss = criterion(output, mel_spec)
    #         val_loss += loss.item() * mel_spec.size(0)
    # val_loss /= len(valid_loader.dataset)

    print(f"Epoch {epoch+1}, train_loss={train_loss:.4f}, val_loss={val_loss:.4f}, Time: {epoch_time:.2f}s")

print("Training complete.")

# --------------------------
# Testing, Inversion, and Audio Playback
# --------------------------
model.eval()

# Samples to compare
test_indices = [0, 1, 2, 3]

for idx in test_indices:
    print(f"\n=== Test sample index: {idx} ===")
    # (mel_spec, sample_rate, key, metadata) from dataset
    mel_spec, sample_rate, key, metadata = test_dataset[idx]

    # 1) Listen to the Original Audio with no transform applied
    raw_waveform, raw_sr = load_raw_waveform("testing", key)
    print(f"Key: {key}")
    print("Original audio:")
    display(Audio(raw_waveform.numpy(), rate=raw_sr))

    # 2) Reconstruct using the model
    mel_spec = mel_spec.unsqueeze(0).to(device)  # shape [1, 1, 64, 128]
    with torch.no_grad():
        reconstructed_mel = model(mel_spec)  # shape [1, 1, 64, 128]

    # 3) Convert the reconstructed mel to waveform
    recon_np = reconstructed_mel.squeeze().cpu().numpy()  # [64, 128]
    # recon_power = librosa.db_to_power(recon_np)  # dB -> power
    reconstructed_audio = librosa.feature.inverse.mel_to_audio(
        recon_np, sr=16000, n_fft=1024, hop_length=501
    )

    print("Reconstructed audio:")
    display(Audio(reconstructed_audio, rate=raw_sr))


PROJECT ROOT is: c:\Users\Articuno\Desktop\TFG-info
Starting training on cuda...


Training:   2%|▏         | 1/50 [00:32<26:09, 32.03s/it]

Epoch 1, train_loss=11.2641, val_loss=0.0000, Time: 32.03s


Training:   4%|▍         | 2/50 [01:03<25:35, 31.98s/it]

Epoch 2, train_loss=11.2165, val_loss=0.0000, Time: 31.95s


...

Reconstructed audio:



=== Test sample index: 2 ===
Key: guitar_acoustic_010-066-100
Original audio:


Reconstructed audio:



=== Test sample index: 3 ===
Key: reed_acoustic_037-068-127
Original audio:


Reconstructed audio:
